In [8]:
import json
from collections import OrderedDict
import numpy as np
import cv2
import os

In [10]:
number = 'SEN0282'
who = 'REAL01'

file_name = f'{number}_{who}_지하철_가방_두다_잊다_3000.json'

In [82]:
os.listdir('./keypoint/real/')

['SEN0241_REAL10_서울_기차_가다_목적_지하철_바꾸다_몇호_원본.json',
 '.DS_Store',
 'SEN0293_REAL10_지하철_환풍구_핸드폰_빠지다_원본.json',
 'SEN0240_REAL10_강남_가다_목적_지하철_바꾸다_몇호_원본.json',
 'SEN0634_REAL10_명동_내리다_원본.json',
 'WORD1082_REAL02_생활교육자_원본.json',
 'SEN0296_REAL01_지하철_문_틈_이어폰_빠지다_원본.json',
 'SEN0281_REAL10_길_지도_보다_찾다_할수있다_원본.json',
 'SEN0020_REAL10_저_원본.json',
 'SEN0221_REAL10_십자로_전_버스_곳_가다_내리다_원본.json',
 'WORD1036_REAL02_행동불량_원본.json',
 'SEN0287_REAL01_어지럽다_쓰러지다_원본.json']

In [81]:
file_name = 'SEN0020_REAL10_저_원본.json'
with open('./keypoint/real/' + file_name) as json_file:
    tacotron_txt = json.load(json_file)

In [5]:
len(tacotron_txt)

190

In [6]:
len(tacotron_txt[0])

255

In [94]:
for real in os.listdir('./request2/'):
    if 'Store' in real:
        continue
#     number = real.split('_')[0]
#     who = real.split('_')[1]
    
    with open('./request2/' + real) as json_file:
        tacotron_txt = json.load(json_file)
        
    json_path = './request2/' + 'result'#+ f'NIA_SL_{number}_{who}_F.mp4'

    os.makedirs(json_path, exist_ok=True)

    cnt = 0
    for file in tacotron_txt:
        pose = file[:30]
        face = file[30:170]
        left_hand = file[170:212]
        right_hand = file[212:-1]

        for _ in range(4):
            pose.insert(20, 0)

        for _ in range(4):
            pose.insert(26, 0)

        for _ in range(12):
            pose.insert(39, 0)

        raw_pose = []
        for i in range(0, len(pose), 2):
            raw_pose.append(pose[i]*2048)
            raw_pose.append(pose[i+1]*1152)
            if pose[i] == 0 or pose[i+1] == 0:
                raw_pose.append(0)
            else:
                raw_pose.append(0.7)

        raw_face = []
        for i in range(0, len(face), 2):
            raw_face.append(face[i]*2048)
            raw_face.append(face[i+1]*1152)
            if face[i] == 0 or face[i+1] == 0:
                raw_face.append(0)
            else:
                raw_face.append(0.7)

        raw_left_hand = []
        for i in range(0, len(left_hand), 2):
            raw_left_hand.append(left_hand[i]*2048)
            raw_left_hand.append(left_hand[i+1]*1152)
            if left_hand[i] == 0 or left_hand[i+1] == 0:
                raw_left_hand.append(0)
            else:
                raw_left_hand.append(0.7)

        raw_right_hand = []
        for i in range(0, len(right_hand), 2):
            raw_right_hand.append(right_hand[i]*2048)
            raw_right_hand.append(right_hand[i+1]*1152)
            if right_hand[i] == 0 or right_hand[i+1] == 0:
                raw_right_hand.append(0)
            else:
                raw_right_hand.append(0.7)



        json_data = OrderedDict()

        json_data['version'] = 1.3
        json_data['people'] = [{
            "person_id":[-1],
            "pose_keypoints_2d":raw_pose,
            "face_keypoints_2d":raw_face,
            "hand_left_keypoints_2d":raw_left_hand,
            "hand_right_keypoints_2d":raw_right_hand,
            "pose_keypoints_3d":[],
            "face_keypoints_3d":[],
            "hand_left_keypoints_3d":[],
            "hand_right_keypoints_3d":[]
        }]

        with open(json_path + f'/NIA_SL_{cnt:012d}_keypoints.json', 'w') as makefile: #f'/NIA_SL_{number}_{who}_F_{cnt:012d}_keypoints.json', 'w') as makefile:
            json.dump(json_data, makefile, ensure_ascii=False, indent='\t')

        cnt += 1
